In [17]:
import pandas as pd
import ast

In [18]:
df_stops_1980 = pd.read_csv('../1980-network/stops_df_1980-final.csv')
df_stops_1982 = pd.read_csv('../1982-network/stops_df_1982-final.csv')

combined_df = pd.concat([df_stops_1980, df_stops_1982], ignore_index=True)

df_1984 = pd.read_csv('stops_df_1984-initial.csv')

In [19]:
df_1984.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,"Moabit, Waldstrasse Ecke Turmstrasse",1,bus,{'1'}
1,1,Turmstrasse,4,bus,"{'70', '1', '16'}"
2,2,Turmstrasse,4,u-bahn,{'9'}


In [20]:
df_1984.drop(columns=['Unnamed: 0'], inplace=True)
df_1984["location"] = ""
df_1984["identifier"] = ""
df_1984["stop_id_other"] = ""
df_1984["previous_in_lines"] = ""
df_1984["stop_description"] = ""

In [21]:
combined_df.head(3)

,Unnamed: 0,stop_id,stop_name,type,location,in_lines,identifier
0,0,19800,Achtermannstrasse,bus,"52.57652468032656, 13.418517187666131",['52'],NaN
1,1,19801,Adalbertstrasse,bus,"52.50548699996121, 13.42221404498021","['78', '75', '78E']",NaN
2,2,19802,Adamstrasse,bus,"52.521029117218866, 13.19780699419913","['97', '34']",NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [22]:
for index, row in df_1984.iterrows():
 
    # and row["in_lines"] is in df_stops_total["in_lines
    condition_in_lines = any(all(value in ast.literal_eval(line) for value in ast.literal_eval(row["in_lines"])) for line in combined_df["in_lines"].values)


    if row["stop_name"] in combined_df["stop_name"].values and condition_in_lines:
        # Find matching rows in df_1960
        matching_rows = combined_df[combined_df["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1984.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1984.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1984.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1984.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1984.at[index, 'location'] = ''
            df_1984.at[index, 'identifier'] = ''
            df_1984.at[index, 'stop_description'] = ''
            df_1984.at[index, 'stop_id_other'] = ''
            df_1984.at[index, 'previous_in_lines'] = ''


In [23]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1984['location'] = df_1984['location'].apply(split_values)
df_1984['identifier'] = df_1984['identifier'].apply(split_values)

In [24]:
df_1984.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [25]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1984 = df_1984.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [26]:
df_1984

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Achtermannstrasse,bus,{'52'},"52.57652468032656, 13.418517187666131",nan,['52']+ {'52'},
1,Adalbertstrasse,bus,"{'75', '78E', '78'}","52.50548699996121, 13.42221404498021",nan,"['78', '75', '78E']+ {'78', '78E'}",
2,Adamstrasse,bus,"{'13', '35', '97', '34'}",,,,
3,Adamstrasse/Wilhelmstrasse,bus,{'92'},"52.5210885080291, 13.18793717048703",nan,{'92'},
4,Adenauer Platz,u-bahn,{'7'},"50.991325,7.130507",Q22692648,{'7'},
...,...,...,...,...,...,...,...
1784,Zweiwinkelweg,bus,{'80'},"52.532774381217585, 13.148393051743144",nan,['80'],
1785,Zwickauer Damm,u-bahn,{'7'},"52.423333333333,13.483888888889",Q572096,['7']+ {'7'},
1786,Zwieseler Strasse,bus,{'94'},"52.49612228648307, 13.540160888423404",nan,{'94'},
1787,u-Bhf. Richard-Wagner-Platz,bus,{'62'},,,,


In [27]:
df_1984.to_csv('1984-stops-combined.csv')